# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Unsupervised Learning Lifecycle - Part 2
**Instructor:** Amir Charkhi | **Focus:** Clustering → Tuning → Evaluation → Insights

---

## 🎯 Where We Are in the Lifecycle

```
┌─────────────────────────────────────────────────────────────────────┐
│                    UNSUPERVISED LEARNING LIFECYCLE                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  PART 1 (Completed):                                               │
│  ┌──────────┐    ┌──────────────┐    ┌─────────────────────┐       │
│  │ Raw Data │ →  │ Preprocessing │ →  │ Dim Reduction (PCA) │  ✓   │
│  └──────────┘    └──────────────┘    └─────────────────────┘       │
│                                                                     │
│  PART 2 (This Notebook):                                           │
│  ┌────────────┐    ┌────────────┐    ┌──────────────────┐          │
│  │ Clustering │ →  │ HP Tuning  │ →  │ Business Insights │  ←      │
│  └────────────┘    └────────────┘    └──────────────────┘          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 📚 What You'll Learn in Part 2

- Compare multiple clustering algorithms
- Systematic hyperparameter tuning
- Evaluate clustering quality
- Select the best model
- Extract actionable business insights

---

## 1. Setup and Load Prepared Data

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

In [ ]:
# Load prepared data from Part 1 (or recreate if not available)
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

wine = load_wine()
X = wine.data
y_true = wine.target
feature_names = wine.feature_names

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca_cluster = PCA(n_components=0.90)
X_pca = pca_cluster.fit_transform(X_scaled)

pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

In [ ]:
f"Data ready: {X_scaled.shape[0]} samples, {X_scaled.shape[1]} original features, {X_pca.shape[1]} PCA features"

---
## 2. The Clustering Workflow

### 🔄 Systematic Approach

```
Step 1: Try multiple algorithms
        K-Means, Hierarchical, DBSCAN, GMM
            ↓
Step 2: For each algorithm, tune hyperparameters
        K-Means: n_clusters = [2, 3, 4, 5, ...]
        DBSCAN: eps, min_samples
            ↓
Step 3: Evaluate each configuration
        Silhouette, Calinski-Harabasz, Davies-Bouldin
            ↓
Step 4: Select best model
        Balance metrics + domain knowledge
            ↓
Step 5: Interpret and apply
        Business insights, action items
```

---

## 3. Evaluation Metrics (No Labels!)

### 📏 Internal Metrics (What We Use)

```
We have NO labels, so we use INTERNAL metrics:

1. Silhouette Score [-1, 1]
   "How similar is each point to its own cluster vs others?"
   Higher = Better

2. Calinski-Harabasz Score [0, ∞]
   "Ratio of between-cluster to within-cluster variance"
   Higher = Better

3. Davies-Bouldin Score [0, ∞]
   "Average similarity between clusters"
   Lower = Better
```

### 🎯 External Metrics (For Validation Only)

```
If we happen to have true labels (like our wine data):

1. Adjusted Rand Index [-1, 1]
   "Agreement between predicted and true clusters"

2. Normalized Mutual Information [0, 1]
   "Information shared between predicted and true"

Note: In real unsupervised learning, we don't have these!
```

---

In [ ]:
def evaluate_clustering(X, labels, y_true=None):
    """Calculate clustering evaluation metrics"""
    results = {
        'Silhouette': silhouette_score(X, labels),
        'Calinski-Harabasz': calinski_harabasz_score(X, labels),
        'Davies-Bouldin': davies_bouldin_score(X, labels)
    }
    
    # External metrics (if true labels available)
    if y_true is not None:
        results['ARI'] = adjusted_rand_score(y_true, labels)
        results['NMI'] = normalized_mutual_info_score(y_true, labels)
    
    return results

---
## 4. Method 1: K-Means with Hyperparameter Tuning

### 🎯 The Key Question: How Many Clusters?

**Approach:** Try multiple values of K, evaluate each

In [ ]:
# Test K from 2 to 8
k_range = range(2, 9)
kmeans_results = []

In [ ]:
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_pca)
    
    metrics = evaluate_clustering(X_pca, labels, y_true)
    metrics['K'] = k
    metrics['Inertia'] = kmeans.inertia_
    kmeans_results.append(metrics)

In [ ]:
df_kmeans = pd.DataFrame(kmeans_results)
df_kmeans.round(3)

### 📊 Visualize K Selection

In [ ]:
fig = make_subplots(rows=2, cols=2,
    subplot_titles=('Elbow Method (Inertia)', 'Silhouette Score', 
                    'Calinski-Harabasz', 'Davies-Bouldin'))

# Elbow
fig.add_trace(go.Scatter(x=list(k_range), y=df_kmeans['Inertia'],
    mode='lines+markers', marker=dict(size=10)), row=1, col=1)

# Silhouette
fig.add_trace(go.Scatter(x=list(k_range), y=df_kmeans['Silhouette'],
    mode='lines+markers', marker=dict(size=10, color='green')), row=1, col=2)

# Calinski-Harabasz
fig.add_trace(go.Scatter(x=list(k_range), y=df_kmeans['Calinski-Harabasz'],
    mode='lines+markers', marker=dict(size=10, color='orange')), row=2, col=1)

# Davies-Bouldin
fig.add_trace(go.Scatter(x=list(k_range), y=df_kmeans['Davies-Bouldin'],
    mode='lines+markers', marker=dict(size=10, color='red')), row=2, col=2)

fig.update_layout(title='K-Means: Hyperparameter Tuning (Finding Optimal K)',
    height=600, width=1000, showlegend=False, template='plotly_white')
fig.update_xaxes(title_text='K', dtick=1)
fig.show()

💡 **Analysis:**
- Elbow: Bend at K=3
- Silhouette: Peak at K=3
- Calinski-Harabasz: Peak at K=3
- Davies-Bouldin: Low at K=3

**Optimal K = 3** (all metrics agree!)

In [ ]:
# Final K-Means model
best_kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_kmeans = best_kmeans.fit_predict(X_pca)

---
## 5. Method 2: Hierarchical Clustering

In [ ]:
# Test different linkage methods
linkages = ['ward', 'complete', 'average']
hier_results = []

In [ ]:
for linkage in linkages:
    for n in range(2, 6):
        hier = AgglomerativeClustering(n_clusters=n, linkage=linkage)
        labels = hier.fit_predict(X_pca)
        
        metrics = evaluate_clustering(X_pca, labels, y_true)
        metrics['Linkage'] = linkage
        metrics['N_Clusters'] = n
        hier_results.append(metrics)

In [ ]:
df_hier = pd.DataFrame(hier_results)
df_hier.sort_values('Silhouette', ascending=False).head()

In [ ]:
# Best hierarchical model
best_hier = AgglomerativeClustering(n_clusters=3, linkage='ward')
labels_hier = best_hier.fit_predict(X_pca)

---
## 6. Method 3: DBSCAN (Density-Based)

In [ ]:
# Test different eps and min_samples combinations
eps_values = [0.5, 1.0, 1.5, 2.0, 2.5]
min_samples_values = [3, 5, 7]
dbscan_results = []

In [ ]:
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_pca)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        if n_clusters >= 2:  # Need at least 2 clusters for metrics
            mask = labels != -1
            metrics = evaluate_clustering(X_pca[mask], labels[mask], y_true[mask])
        else:
            metrics = {'Silhouette': np.nan, 'Calinski-Harabasz': np.nan, 
                      'Davies-Bouldin': np.nan, 'ARI': np.nan, 'NMI': np.nan}
        
        metrics['eps'] = eps
        metrics['min_samples'] = min_samples
        metrics['N_Clusters'] = n_clusters
        metrics['N_Noise'] = n_noise
        dbscan_results.append(metrics)

In [ ]:
df_dbscan = pd.DataFrame(dbscan_results)
df_dbscan[df_dbscan['N_Clusters'] >= 2].sort_values('Silhouette', ascending=False).head()

In [ ]:
# Best DBSCAN model
best_dbscan = DBSCAN(eps=2.0, min_samples=5)
labels_dbscan = best_dbscan.fit_predict(X_pca)

---
## 7. Method 4: Gaussian Mixture Model

In [ ]:
# Test different numbers of components
gmm_results = []

for n in range(2, 7):
    gmm = GaussianMixture(n_components=n, random_state=42, n_init=5)
    labels = gmm.fit_predict(X_pca)
    
    metrics = evaluate_clustering(X_pca, labels, y_true)
    metrics['N_Components'] = n
    metrics['BIC'] = gmm.bic(X_pca)  # Lower is better
    metrics['AIC'] = gmm.aic(X_pca)  # Lower is better
    gmm_results.append(metrics)

In [ ]:
df_gmm = pd.DataFrame(gmm_results)
df_gmm.round(3)

In [ ]:
# Best GMM model
best_gmm = GaussianMixture(n_components=3, random_state=42, n_init=5)
labels_gmm = best_gmm.fit_predict(X_pca)

---
## 8. Compare All Methods

### 📊 Side-by-Side Visualization

In [ ]:
# Create comparison plot
fig = make_subplots(rows=2, cols=2,
    subplot_titles=('K-Means (K=3)', 'Hierarchical (Ward)', 
                    'DBSCAN', 'Gaussian Mixture'))

all_labels = [labels_kmeans, labels_hier, labels_dbscan, labels_gmm]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for labels, (row, col) in zip(all_labels, positions):
    fig.add_trace(
        go.Scatter(
            x=X_pca_2d[:, 0], y=X_pca_2d[:, 1],
            mode='markers',
            marker=dict(color=labels, colorscale='viridis', size=8, opacity=0.7),
            showlegend=False
        ),
        row=row, col=col
    )

fig.update_layout(title='Clustering Methods Comparison',
    height=700, width=900, template='plotly_white')
fig.show()

### 📈 Metrics Comparison

In [ ]:
# Collect best results from each method
comparison = [
    {'Method': 'K-Means', **evaluate_clustering(X_pca, labels_kmeans, y_true)},
    {'Method': 'Hierarchical', **evaluate_clustering(X_pca, labels_hier, y_true)},
    {'Method': 'GMM', **evaluate_clustering(X_pca, labels_gmm, y_true)}
]

# Add DBSCAN (excluding noise points for fair comparison)
mask = labels_dbscan != -1
if mask.sum() > 10:
    comparison.append({
        'Method': 'DBSCAN',
        **evaluate_clustering(X_pca[mask], labels_dbscan[mask], y_true[mask])
    })

In [ ]:
df_comparison = pd.DataFrame(comparison)
df_comparison.round(3)

In [ ]:
# Visualize comparison
fig = go.Figure()

metrics_to_plot = ['Silhouette', 'ARI', 'NMI']
colors = ['steelblue', 'green', 'orange']

for i, metric in enumerate(metrics_to_plot):
    fig.add_trace(go.Bar(
        name=metric,
        x=df_comparison['Method'],
        y=df_comparison[metric],
        marker_color=colors[i]
    ))

fig.update_layout(
    title='Clustering Performance Comparison (Higher is Better)',
    xaxis_title='Method',
    yaxis_title='Score',
    barmode='group',
    template='plotly_white',
    height=500, width=800
)
fig.show()

---
## 9. Select Best Model

In [ ]:
# Find best method by average rank
df_ranks = df_comparison.copy()
df_ranks['Silhouette_Rank'] = df_ranks['Silhouette'].rank(ascending=False)
df_ranks['ARI_Rank'] = df_ranks['ARI'].rank(ascending=False)
df_ranks['NMI_Rank'] = df_ranks['NMI'].rank(ascending=False)
df_ranks['DB_Rank'] = df_ranks['Davies-Bouldin'].rank(ascending=True)  # Lower is better

df_ranks['Avg_Rank'] = df_ranks[['Silhouette_Rank', 'ARI_Rank', 'NMI_Rank', 'DB_Rank']].mean(axis=1)
df_ranks[['Method', 'Avg_Rank']].sort_values('Avg_Rank')

### 🏆 Winner Selection

In [ ]:
best_method = df_ranks.loc[df_ranks['Avg_Rank'].idxmin(), 'Method']
f"🏆 Best Method: {best_method}"

In [ ]:
# Use best method's labels
final_labels = labels_gmm  # GMM typically performs well on this dataset

---
## 10. Business Insights: What We Get Out

### 🍷 Wine Segment Profiles

In [ ]:
# Create DataFrame with clusters
df_wine = pd.DataFrame(X, columns=feature_names)
df_wine['Cluster'] = final_labels

In [ ]:
# Calculate cluster profiles
cluster_profiles = df_wine.groupby('Cluster').mean().round(2)
cluster_profiles

In [ ]:
# Cluster sizes
df_wine['Cluster'].value_counts().sort_index()

### 📊 Key Differentiating Features

In [ ]:
# Calculate how much each feature differs between clusters
feature_variance = cluster_profiles.std().sort_values(ascending=False)

fig = px.bar(
    x=feature_variance.values,
    y=feature_variance.index,
    orientation='h',
    title='Feature Importance: What Differentiates Wine Segments?',
    labels={'x': 'Variance Across Clusters', 'y': 'Feature'},
    template='plotly_white',
    height=500, width=800
)
fig.show()

### 🎯 Segment Interpretation

In [ ]:
# Top features comparison
top_features = ['proline', 'color_intensity', 'flavanoids', 'alcohol', 'od280/od315_of_diluted_wines']

fig = px.bar(
    cluster_profiles[top_features].reset_index().melt(id_vars='Cluster'),
    x='variable', y='value', color='Cluster',
    barmode='group',
    title='Top 5 Differentiating Features by Segment',
    labels={'variable': 'Feature', 'value': 'Average Value'},
    template='plotly_white',
    height=500, width=900
)
fig.show()

### 📝 Business Summary

In [ ]:
# Create business-friendly segment names
segment_names = {
    0: 'Premium Reserve',
    1: 'Classic Selection',
    2: 'Bold & Rich'
}

df_wine['Segment'] = df_wine['Cluster'].map(segment_names)

In [ ]:
# Final visualization with segment names
df_plot = df_wine.copy()
df_plot['PC1'] = X_pca_2d[:, 0]
df_plot['PC2'] = X_pca_2d[:, 1]

fig = px.scatter(
    df_plot, x='PC1', y='PC2',
    color='Segment',
    title='Final Wine Segmentation',
    template='plotly_white',
    width=900, height=600,
    color_discrete_sequence=['#e41a1c', '#377eb8', '#4daf4a']
)
fig.update_traces(marker=dict(size=10, opacity=0.8))
fig.show()

---
## 11. Actionable Recommendations

### 🍷 Segment Characteristics

**Segment 0: Premium Reserve**
```
• High proline and flavanoids
• Lower color intensity
• Higher OD280/OD315 ratio
• Position: Ultra-premium, collectors
• Price: $$$
```

**Segment 1: Classic Selection**
```
• Moderate all characteristics
• Balanced profile
• Position: Everyday premium
• Price: $$
```

**Segment 2: Bold & Rich**
```
• High color intensity
• Higher alcohol content
• Lower flavanoids
• Position: Bold flavor seekers
• Price: $$-$$$
```

### 💼 Business Actions

```
1. MARKETING
   • Target different segments with tailored messaging
   • Premium Reserve → Luxury magazines, collectors
   • Classic Selection → General wine lovers
   • Bold & Rich → Steak houses, BBQ enthusiasts

2. PRICING
   • Adjust price points based on segment
   • Premium Reserve commands higher margins

3. INVENTORY
   • Stock based on segment popularity
   • Classic Selection likely highest volume

4. RECOMMENDATIONS
   • Build recommendation engine using clusters
   • "If you liked X, try Y from same segment"
```

---

## 12. Complete Lifecycle Summary

### ✅ The Full Pipeline

```
┌─────────────────────────────────────────────────────────────────────┐
│                    COMPLETE UNSUPERVISED PIPELINE                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. DATA PREPARATION                                                │
│     • Load raw data (178 wines × 13 features)                       │
│     • EDA: distributions, correlations                              │
│     • Preprocessing: StandardScaler                                 │
│                                                                     │
│  2. DIMENSIONALITY REDUCTION                                        │
│     • PCA: 13 → 6 features (90% variance)                          │
│     • t-SNE: For visualization only                                 │
│                                                                     │
│  3. CLUSTERING                                                      │
│     • K-Means: K = [2-8], best K=3                                 │
│     • Hierarchical: linkage=[ward,complete,average]                 │
│     • DBSCAN: eps=[0.5-2.5], min_samples=[3-7]                     │
│     • GMM: n_components=[2-6]                                       │
│                                                                     │
│  4. EVALUATION                                                      │
│     • Internal: Silhouette, CH, DB                                  │
│     • External: ARI, NMI (validation only)                          │
│                                                                     │
│  5. MODEL SELECTION                                                 │
│     • Compare all methods on all metrics                            │
│     • Rank and select best                                          │
│                                                                     │
│  6. BUSINESS OUTPUT                                                 │
│     • Segment profiles                                              │
│     • Feature importance                                            │
│     • Actionable recommendations                                    │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 Key Takeaways

**1. Unsupervised Learning Lifecycle**
- Not just "apply algorithm" - it's a systematic process
- Data prep → Dim reduction → Clustering → Evaluation → Insights

**2. Hyperparameter Tuning is Critical**
- Every algorithm has parameters to tune
- Test multiple values systematically
- Use metrics to guide selection

**3. Multiple Methods, Compare Results**
- No single "best" algorithm
- Different methods may find different patterns
- Compare on multiple metrics

**4. Domain Knowledge Matters**
- Metrics guide, but don't decide
- Interpret clusters in business context
- Actionable insights are the goal

**5. The Output is Business Value**
- Clusters alone are not valuable
- Profiles, segments, recommendations matter
- Connect to business actions

---

**Congratulations! You've completed the full unsupervised learning lifecycle!** 🎉

---

**AI Tech Institute** | *Building Tomorrow's AI Engineers Today*